# Import and Read data


In [9]:
import sys
sys.path.append('.')  # Add current directory to path
from build_graph_and_train import *

In [10]:
partition = 478

In [11]:
df = pd.read_csv(f"../../../data/top30groups/LongLatCombined/combined/combined{partition}.csv")

In [12]:
from sklearn.preprocessing import StandardScaler

# Columns to exclude from scaling
exclude_cols = ['gname', 'longitude', 'latitude']

# Columns to scale
scale_cols = [col for col in df.columns if col not in exclude_cols]

# Scale only selected columns
scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])

In [13]:
import os 
if not os.path.isdir(f"Results{partition}"):
    os.mkdir(f"Results{partition}")

# Create longlat feature

In [14]:
geodata = ['longitude', 'latitude']
combined_geo = df.copy()
combined_geo['longlat'] = list(zip(df['longitude'], df['latitude']))
combined_geo = combined_geo.drop(columns=geodata)

In [15]:
import ast

def to_tuple_if_needed(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val  # already a tuple

combined_geo['longlat'] = combined_geo['longlat'].apply(to_tuple_if_needed)

# Weapon type prediction

In [16]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_cols = ['weaptype1', 'nkill', 'targtype1', 'attacktype1']
y_preds = []
y_trues = []
logs = []
for i in range(len(continuous_cols)):
    data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index, index_to_label = build_graph_data(combined_geo, label_index, continuous_col=continuous_cols[i])
    args = {
        'partition': f"gtd{partition}",
        'embed_dim': 16,
        'lr': 0.01,
        'epochs': 300,
        'feat_dropout': 0,
        'n_tree': 80,
        'tree_depth': 10,
        'tree_feature_rate': 0.5,
        'n_class': len(label_index)
    }
    best_acc, best_epoch, best_precision, best_recall, best_f1, y_pred_decoded, y_true_decoded, best_precision_micro, best_recall_micro, best_f1_micro, best_precision_macro, best_recall_macro, best_f1_macro, roc_auc_weighted, roc_auc_micro, roc_auc_macro, epoch_logs
    y_preds.append(y_pred_decoded)
    y_trues.append(y_true_decoded)
    logs.append(epoch_logs)
    
    with open(f"Results{partition}/Results_{continuous_cols[i]}_prediction", "w") as f:
        f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best recall weighted: {best_recall} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best precision weighted: {best_precision} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best f1 weighted: {best_f1} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best recall macro: {best_recall_macro} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best precision macro: {best_precision_macro} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best f1 macro: {best_f1_macro} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best recall micro: {best_recall_micro} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best precision micro: {best_precision_micro} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best f1 micro: {best_f1_micro} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")

        f.write(f"Best auroc weighted: {roc_auc_weighted} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best auroc micro: {roc_auc_micro} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best auroc macro: {roc_auc_macro} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")

/home/jovyan/MEX0512/GTD_2025/Codes/Baselines/GraphRfi_OneHotLongLat/build_graph_and_train.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


RuntimeError: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "../c10/cuda/CUDACachingAllocator.cpp":844, please report a bug to PyTorch. 

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_pred_decoded, y_true_decoded))

In [ ]:
def plot_confusion_matrix(y_true, y_pred, labels, continuous_col):
    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Save the figure
    save_path = f"Results{partition}/cm_{partition}_{continuous_col}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition {partition} to {save_path}")


In [ ]:
for i in range(len(continuous_cols)):
    plot_confusion_matrix(y_preds[i], y_trues[i], sorted(df['gname'].unique()), continuous_cols[i])

Saved confusion matrix for partition 100 to Results100/cm_100_weaptype1.png
Saved confusion matrix for partition 100 to Results100/cm_100_nkill.png
Saved confusion matrix for partition 100 to Results100/cm_100_targtype1.png
Saved confusion matrix for partition 100 to Results100/cm_100_attacktype1.png
